In [ ]:
# Self-Playing Agent
# When the agent does self-play, the states seen by the agent is different when it plays first move, and when it plays second move.
# Let's call the agentFirst, and agentSecond as the valueFunctions related to the agent when it plays first and second respectively.
# During the firstRound the moves are made randomly, but the valueFunction related to during which play agent won, this agent's valueFunction will be updated.
# Simlarly either during exploration or randomness the agent's first or second turn related valueFunctions are updated based on who won.
# This learning continues to move forward learning moves, correcting mistakes, improving valueFunction. The agent will self-learn.

In [ ]:
# Tic-Tac-Toe game
# create an opponent, ie, policy with is the brain of the opponent.
# should we consider all random policies of the opponent, our learning based on TD will be easy, but it will increase noise for learning. (maybe or maynot be), Test out both the cases.
# I am the opponent, I will get a chance to play with probability 2/3, or else random action is taken.

# Do we have any reward, where are we getting reward from to update the value function in temporal difference, or here is it delayed reward

In [1]:
import numpy as np
import random
import copy

In [13]:
# The player who starts the game plays with X
# environment includes the opponent player
# makeMove function to interact with the environment, return the updated environment.
# The opponent is part of the environment.

# Instead of iterating over all the possible moves everytime, just update the possible moves inside the environment
class TicTacToe():
    def __init__(self, opponentRandomness):
        self.state = [['.' for i in range(3)] for i in range(3)]
        self.possibleMoves = [[i, j] for i in range(3) for j in range(3)]
        self.terminated = False
        self.opponentRandomness = opponentRandomness

    def checkGame(self):
        if len(self.possibleMoves) == 0:
            self.terminated = True
            return "Draw"

        for i in range(3):
            if self.state[i][0]==self.state[i][1] and self.state[i][1]==self.state[i][2] and self.state[i][0]!='.':
                self.terminated = True
                return f"{self.state[i][0]} is winner"

            if self.state[0][i]==self.state[1][i] and self.state[1][i]==self.state[2][i] and self.state[0][i]!='.':
                self.terminated = True
                return f"{self.state[0][i]} is winner"

        if self.state[0][0]==self.state[1][1] and self.state[1][1]==self.state[2][2] and self.state[1][1]!='.':
            self.terminated = True
            return f"{self.state[0][0]} is winner"

        if self.state[2][0]==self.state[1][1] and self.state[1][1]==self.state[0][2] and self.state[1][1]!='.':
            self.terminated = True
            return f"{self.state[1][1]} is winner"
        return

    def printState(self, state):
        print("current state: ")
        for i in range(3):
            for j in range(3):
                print(state[i][j], end=" ")
            print()

    def opponentMove(self):
        # opponent make the move randomly
        if np.random.rand() < self.opponentRandomness :
            print("opponent made a random move")
            move = random.choice(self.possibleMoves)
        # opponent choose the move
        else:
            print(f"Choose moves from these possible moves {self.possibleMoves}")
            while True:
                try:
                    move = list(map(int, input("Enter your move ").split()))
                    assert(move in self.possibleMoves)
                    break
                except:
                    print("Invalid Move")
        # updating the possible moves properly
        self.possibleMoves.remove(move)
        print(f"Opponent made move {move}")
        self.state[move[0]][move[1]] = 'O'
        self.printState(self.state)
        self.checkGame()
        return

    def makeMove(self, i, j):
        if self.terminated:
            raise Exception("GAME ENDED, please reset it")
        print(f"Agent made move [{i}][{j}]")
        if [i, j] not in self.possibleMoves:
            raise Exception("Invalid Move")
        self.state[i][j] = 'X'
        self.possibleMoves.remove([i, j])
        self.printState(self.state)
        self.checkGame()
        if not self.terminated:
            self.opponentMove()
        return

    def resetGame(self):
        self.state = [['.' for i in range(3)] for i in range(3)]
        self.terminated = False
        self.possibleMoves = [[i, j] for i in range(3) for j in range(3)]
        print("Resetting the Game")
        print("You can now start new game/episode")

In [14]:
# when agent plays first, all states encountered by the agent will be filled at even spots.
# when agent plays second, all states encountered by the agent will be filled at odd spots.
# we can keep a shared valueFunction/ valueTable togther whether agent starts first or second.
# Let's assume agent plays X always, for simplicity.
class Agent():
    def __init__(self, beta=0.99):
        self.valueFunction = {}
        self.previousState = [['.' for i in range(3)] for i in range(3)]
        self.beta = beta

    def shrink(self, state):
        s = ""
        for i in range(3):
            for j in range(3):
                s += str(state[i][j])
        return s

    def initialStateValue(self, state):
        "Will be used once on each state, asses the starting value of each state"
        # Value of game win/lose states
        # 1. if winner is X -> 1
        # 2. if winnder is O -> 0
        for i in range(3):
            if state[i][0]==state[i][1] and state[i][1]==state[i][2] and state[i][0]!='.':
                return state[i][0]=='X'

            if state[0][i]==state[1][i] and state[1][i]==state[2][i] and state[0][i]!='.':
                return state[0][i]=='X'

        if state[0][0]==state[1][1] and state[1][1]==state[2][2] and state[1][1]!='.':
            return state[0][0]=='X'
        if state[2][0]==state[1][1] and state[1][1]==state[0][2] and state[1][1]!='.':
            return state[1][1]=='X'

        # Value of draw state is 0, and anyother state which is not win or lose is 0.5 initially
        for i in range(3):
            for j in range(3):
                if state[i][j]=='.':
                    return 0.5
        return 0

    def getValue(self, state):
        if self.shrink(state) in self.valueFunction.keys():
            return self.valueFunction[self.shrink(state)]
        else:
            self.valueFunction[self.shrink(state)] = self.initialStateValue(state)
            return self.valueFunction[self.shrink(state)]

    def updateValue(self, nextState):
        print(self.previousState)
        self.getValue(self.previousState)
        self.valueFunction[self.shrink(self.previousState)] += self.beta*(self.getValue(nextState) - self.getValue(self.previousState))
        self.previousState = copy.deepcopy(nextState)
        print(f"value is {self.valueFunction[self.shrink(self.previousState)]}")
        self.beta = self.beta * self.beta
        return

    def chooseMove(self, game):
        # Exploration
        if np.random.rand() > self.beta:
            # no temporal difference update for exploration, this is not following our policy or value function
            return random.choice(game.possibleMoves)
        else:
        # Exploitation
            optimalMove = [-1, -1]
            optimalQValue = 0

            for move in game.possibleMoves:
                # This is kindoff getting Qvalue of currentState, action
                nextState = copy.deepcopy(game.state)
                nextState[move[0]][move[1]] = 'X'
                if self.getValue(nextState) >= optimalQValue:
                    optimalMove = move
                    optimalValue = self.getValue(nextState)
                    optimalState = copy.deepcopy(nextState)

            # temporal difference update, based on the state we moved to update the current state.
            # if there are no possible moves, the nextState and previousState are still different, but our agent will try to update some jargon values, won't make much difference.
            self.updateValue(nextState)
            print(nextState)
            return optimalMove


In [15]:
numOfGames = 10000
agent = Agent()
results = []
game = TicTacToe(1)
# Observation: Playing against a player whose moves are choosen randomly, the convergence is low, but it is able to learn to win. But not all the times
# After 2e4 rounds, it was able to win 2/3 times against a randomised play player.
# Quality of the opponent play matters for learning, looks like sample efficient only. Don't really understand why it is sample inefficient.
# It would have
for gg in range(numOfGames):
    while not game.terminated:
        move = agent.chooseMove(game)
        game.makeMove(move[0], move[1])
    results.append(game.checkGame())
    game.resetGame()

Streaming output truncated to the last 5000 lines.
. O O 
X . . 
X . X 
opponent made a random move
Opponent made move [2, 1]
current state: 
. O O 
X . . 
X O X 
Agent made move [0][0]
current state: 
X O O 
X . . 
X O X 
Resetting the Game
You can now start new game/episode
Agent made move [0][2]
current state: 
. . X 
. . . 
. . . 
opponent made a random move
Opponent made move [0, 1]
current state: 
. O X 
. . . 
. . . 
Agent made move [1][1]
current state: 
. O X 
. X . 
. . . 
opponent made a random move
Opponent made move [2, 1]
current state: 
. O X 
. X . 
. O . 
Agent made move [2][2]
current state: 
. O X 
. X . 
. O X 
opponent made a random move
Opponent made move [1, 0]
current state: 
. O X 
O X . 
. O X 
Agent made move [0][0]
current state: 
X O X 
O X . 
. O X 
Resetting the Game
You can now start new game/episode
Agent made move [1][2]
current state: 
. . . 
. . X 
. . . 
opponent made a random move
Opponent made move [0, 0]
current state: 
O . . 
. . X 
. . . 
Agent

In [10]:
print(results)

['X is winner', 'Draw', 'Draw', 'O is winner', 'Draw', 'X is winner', 'X is winner', 'Draw', 'O is winner', '. is winner', 'X is winner', 'X is winner', 'Draw', 'Draw', 'O is winner', 'X is winner', 'Draw', 'X is winner', 'Draw', 'X is winner', 'X is winner', 'X is winner', '. is winner', 'Draw', 'Draw', 'X is winner', 'X is winner', 'O is winner', 'O is winner', 'O is winner', 'X is winner', 'O is winner', 'O is winner', 'Draw', 'O is winner', 'Draw', 'O is winner', 'X is winner', 'X is winner', 'X is winner', 'X is winner', 'X is winner', 'O is winner', 'O is winner', 'O is winner', 'Draw', 'O is winner', 'O is winner', 'Draw', 'O is winner', 'X is winner', 'Draw', 'X is winner', 'Draw', 'O is winner', 'O is winner', 'X is winner', 'Draw', '. is winner', 'X is winner', 'X is winner', 'X is winner', 'Draw', 'X is winner', 'X is winner', 'X is winner', 'Draw', 'Draw', 'Draw', 'Draw', 'X is winner', 'Draw', 'X is winner', '. is winner', 'Draw', 'O is winner', 'O is winner', '. is winner

In [ ]:
results = {}
for gg in range(numOfGames):
    while not game.terminated:
        move = agent.chooseMove(game)
        game.makeMove(move[0], move[1])
    results[game.checkGame()] = results.get(game.checkGame(), 0)+1
    game.resetGame()

print(results)

In [23]:
# Observation: If I'm playing a same move continously, the agent learns to block my moves very quickly.
game = TicTacToe(0)
while not game.terminated:
    move = agent.chooseMove(game)
    game.makeMove(move[0], move[1])

Agent made move [0][1]
current state: 
. X . 
. . . 
. . . 
Choose moves from these possible moves [[0, 0], [0, 2], [1, 0], [1, 1], [1, 2], [2, 0], [2, 1], [2, 2]]
Enter your move 0 0
Opponent made move [0, 0]
current state: 
O X . 
. . . 
. . . 
Agent made move [0][2]
current state: 
O X X 
. . . 
. . . 
Choose moves from these possible moves [[1, 0], [1, 1], [1, 2], [2, 0], [2, 1], [2, 2]]
Enter your move 1 0
Opponent made move [1, 0]
current state: 
O X X 
O . . 
. . . 
Agent made move [2][0]
current state: 
O X X 
O . . 
X . . 
Choose moves from these possible moves [[1, 1], [1, 2], [2, 1], [2, 2]]
Enter your move 1 1
Opponent made move [1, 1]
current state: 
O X X 
O O . 
X . . 
Agent made move [2][2]
current state: 
O X X 
O O . 
X . X 
Choose moves from these possible moves [[1, 2], [2, 1]]
Enter your move 2 1
Opponent made move [2, 1]
current state: 
O X X 
O O . 
X O X 
Agent made move [1][2]
current state: 
O X X 
O O X 
X O X 
